In [1]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Read resources
CA_videos = pd.read_csv('Resources/CAvideos.csv')
US_videos = pd.read_csv('Resources/USvideos.csv')
CA_category_id = pd.read_json('Resources/CA_category_id.json')
US_category_id = pd.read_json('Resources/US_category_id.json')

In [3]:
#Jumaan start here

In [4]:
#Check the columns on US_videos
US_videos.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [5]:
#Drop unnecessary columns
US_videos_new = US_videos[['title', 'channel_title', 'category_id', 'views', 'likes', 'dislikes', 'comment_count']]
US_videos_new.head()

,title,channel_title,category_id,views,likes,dislikes,comment_count
0,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,748374,57527,2966,15954
1,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2418783,97185,6146,12703
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,3191434,146033,5339,8181
3,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,343168,10172,666,2146
4,I Dare You: GOING BALD!?,nigahiga,24,2095731,132235,1989,17518


In [6]:
#Check for duplicates and drop any duplicate rows
print(len(US_videos_new)) 
print(len(US_videos_new['title']))

40949
40949


In [7]:
#Check the columns on US_category_id
US_category_id.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [8]:
#Turn items column into a list
US_category_id_list = US_category_id['items'].tolist()

#Loop through the list and extract category id and title
category_id = []
category_title = []

for category in US_category_id_list:
    category_id.append(int(category['id']))
    category_title.append(category['snippet']['title'])

#Store the information in a datagrame
US_categories = pd.DataFrame({"category_title": category_title, "category_id": category_id})

US_categories.head()

,category_title,category_id
0,Film & Animation,1
1,Autos & Vehicles,2
2,Music,10
3,Pets & Animals,15
4,Sports,17


In [9]:
#Merge both dataframes on category_id
US_merged = pd.merge(US_videos_new, US_categories, on="category_id", how="inner")
US_merged.head()

,title,channel_title,category_id,views,likes,dislikes,comment_count,category_title
0,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,748374,57527,2966,15954,People & Blogs
1,Me-O Cats Commercial,Nobrand,22,98966,2486,184,532,People & Blogs
2,"AFFAIRS, EX BOYFRIENDS, $18MILLION NET WORTH -...",Shawn Johnson East,22,321053,4451,1772,895,People & Blogs
3,BLIND(folded) CAKE DECORATING CONTEST (with Mo...,Grace Helbig,22,197062,7250,217,456,People & Blogs
4,Wearing Online Dollar Store Makeup For A Week,Safiya Nygaard,22,2744430,115426,1110,6541,People & Blogs


In [10]:
#Jumaan end here

In [11]:
#Shrey start here

In [12]:
#Reading the CAvideo file into a dataframe
CA_videos.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


In [13]:
#Dropping the unnecessary columns
CA_videos_new = CA_videos[["title","category_id","views","likes","dislikes","comment_count"]].copy()
CA_videos_new.head()

,title,category_id,views,likes,dislikes,comment_count
0,Eminem - Walk On Water (Audio) ft. Beyoncé,10,17158579,787425,43420,125882
1,PLUSH - Bad Unboxing Fan Mail,23,1014651,127794,1688,13030
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",23,3191434,146035,5339,8181
3,I Dare You: GOING BALD!?,24,2095828,132239,1989,17518
4,Ed Sheeran - Perfect (Official Music Video),10,33523622,1634130,21082,85067


In [14]:
#Renaming the columns
CA_videos_new.rename(columns={"comment_count":"comments"},inplace= True)
CA_videos_new.head()

,title,category_id,views,likes,dislikes,comments
0,Eminem - Walk On Water (Audio) ft. Beyoncé,10,17158579,787425,43420,125882
1,PLUSH - Bad Unboxing Fan Mail,23,1014651,127794,1688,13030
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",23,3191434,146035,5339,8181
3,I Dare You: GOING BALD!?,24,2095828,132239,1989,17518
4,Ed Sheeran - Perfect (Official Music Video),10,33523622,1634130,21082,85067


In [15]:
len(CA_videos_new["title"].unique())

24573

In [16]:
len(CA_videos_new)

40881

In [17]:
#Dropping the duplicates
CA_videos_new.drop_duplicates(subset=["title"],keep = "last",inplace=True)

In [18]:
len(CA_videos_new)

24573

In [19]:
CA_category_id.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
1,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
2,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
3,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
4,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."


In [20]:
len(CA_category_id)

31

In [21]:
CA_category_id["items"][0]

{'kind': 'youtube#videoCategory',
 'etag': '"ld9biNPKjAjgjV7EZ4EKeEGrhao/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
 'id': '1',
 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title': 'Film & Animation',
  'assignable': True}}

In [22]:
items_list = CA_category_id["items"].tolist()

In [23]:
#Extracting the required information from the category dataframe
id_list = []
title=  []
for item in items_list:
    id_list.append(int(item["id"]))
    title.append(item["snippet"]["title"])

In [24]:
#Storing the information in a dataframe
new_category_df = pd.DataFrame({"category_title":title,"category_id":id_list})

In [25]:
new_category_df.head()

,category_title,category_id
0,Film & Animation,1
1,Autos & Vehicles,2
2,Music,10
3,Pets & Animals,15
4,Sports,17


In [26]:
len(new_category_df)

31

In [27]:
new_category_df.dtypes

category_title    object
category_id        int64
dtype: object

In [28]:
new_category_df["category_id"] = new_category_df["category_id"].astype(int)
new_category_df.dtypes

category_title    object
category_id        int32
dtype: object

In [29]:
#Merging the individual country dataframes with the category dataframe
mergedCA_df = pd.merge(CA_videos_new,new_category_df,on="category_id",how="inner")
mergedCA_df.tail()

,title,category_id,views,likes,dislikes,comments,category_title
24517,"Priyamanaval Episode 1036, 08/06/18",43,565220,3669,1188,561,Shows
24518,"Priyamanaval Episode 1038, 11/06/18",43,613807,4058,1084,666,Shows
24519,"Priyamanaval Episode 1039, 12/06/18",43,565091,3581,1408,710,Shows
24520,"Priyamanaval Episode 1040, 13/06/18",43,535254,3105,1403,898,Shows
24521,Golak Bugni Bank Te Batua Full Movie (HD) | Ha...,30,5661965,67232,3784,3851,Movies


In [30]:
mergedUS_df = pd.merge(CA_videos_new,new_category_df,on="category_id",how="inner")
mergedUS_df.head()

,title,category_id,views,likes,dislikes,comments,category_title
0,Jake Paul Says Alissa Violet CHEATED with LOGA...,25,1309699,103755,4613,12143,News & Politics
1,WOW NAPAKAHUSAY ! DUTERTE AT TRUMP NAPAHANGA S...,25,234983,2358,125,344,News & Politics
2,"The National for Sunday, November 12, 2017",25,13433,74,57,0,News & Politics
3,Final Nail In The Coffin: SNL Turns Against Th...,25,116344,5068,106,1341,News & Politics
4,Why the rise of the robots won’t mean the end ...,25,256426,12656,1363,2368,News & Politics


In [31]:
#Anji start here

In [ ]:
#Anji end here